In [1]:
GENIUS_API_TOKEN='genius_credentials'

In [9]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

In [10]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
                #print(hit['result']['url'])
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
morlockk = request_song_url('Morlockk Dilemma', 2)

Found 2 songs by Morlockk Dilemma


In [11]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics

In [12]:
# Get artist object from Genius API
def request_song_info(song_title, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': song_title}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# Get song url form Genius API
def scrape_lyrics_genre(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = lyrics.replace('\n', ' ')
    
    #genre = html.find('a', {'class': 'metadata_with_icon-tags-primary_tag'})
    return lyrics #, genre


# Get Genius.com lyrics from song object
def request_song_lyrics(song_title, artist_name):
    page = 1
    
    response = request_song_info(song_title, page)
    json = response.json()

    # Collect song objects
    song_info = {}
    song = json['response']['hits'][0]['result']
    if song_title.lower() in song['title'].lower() and artist_name.lower() in song['primary_artist']['name'].lower():
        url = song['url']
        lyrics = scrape_lyrics_genre(url)
        song_info[song_title] = lyrics
        
    return song_info
    
# DEMO
mamma = request_song_lyrics('Mamma mia', 'ABBA')

In [13]:
mamma

{'Mamma mia': "   I've been cheated by you since I don't know when So I made up my mind, it must come to an end Look at me now, will I ever learn? I don't know how, but I suddenly lose control There's a fire within my soul   Just one look and I can hear a bell ring One more look and I forget everything, woah   Mamma mia, here I go again My my, how can I resist you? Mamma mia, does it show again? My my, just how much I've missed you Yes, I've been brokenhearted Blue since the day we parted Why, why did I ever let you go? Mamma mia, now I really know My my, I could never let you go   I've been angry and sad about things that you do I can't count all the times that I've told you we're through And when you go, when you slam the door I think you know that you won't be away too long You know that I'm not that strong   Just one look and I can hear a bell ring One more look and I forget everything, woah   Mamma mia, here I go again My my, how can I resist you? Mamma mia, does it show again? My